In [1]:
# 📓 test_generate_report_zip.ipynb

# Paso 1 - Configuración inicial
import os
import sys
import pandas as pd
import zipfile

# Ajustar el sys.path para que encuentre tus módulos
sys.path.append(r'C:\Desarrollo\strix-system')

# Importar UnitOfWork, IncidentService y PDFReportAdapter
from core.infraestructure.uow.sqlalchemy_uow import SQLAlchemyUnitOfWork
from reports.incident.services.incident_service import IncidentService
from reports.incident.adapters import PDFReportAdapter

# Crear UnitOfWork e IncidentService
uow = SQLAlchemyUnitOfWork()
incident_service = IncidentService(uow)

# Paso 2 - Recuperar incidentes
with uow:
    rows = incident_service.get_all_incidents()
    df = pd.DataFrame(rows)

# Validar si hay datos
if df.empty:
    print("⚠️ No hay registros verificados para generar informe.")
else:
    print(f"✅ {len(df)} registros recuperados para el reporte.")

    # Paso 3 - Crear el PDF
    output_dir = os.path.join(os.getcwd(), "output")
    os.makedirs(output_dir, exist_ok=True)
    pdf_path = os.path.join(output_dir, "informe_verificacion_siniestros.pdf")

    PDFReportAdapter.build(df.copy(), pdf_path)
    print(f"✅ PDF generado: {pdf_path}")

    



✅ Conexión a S3 establecida con éxito.
✅ Nueva Conexión a PostgreSQL establecida.
🚨 ENTER: Se abrió una nueva conexión SQLAlchemy
🚨 ENTER: Se abrió una nueva conexión SQLAlchemy
✅ 2 registros recuperados para el reporte.
✅ Informe generado exitosamente en c:\Desarrollo\strix-system\reports\incident\test\integration\output\informe_verificacion_siniestros.pdf
✅ PDF generado: c:\Desarrollo\strix-system\reports\incident\test\integration\output\informe_verificacion_siniestros.pdf


In [2]:
from reports.incident.adapters.excel_report_adapter import ExcelReportAdapter

excel_path = ExcelReportAdapter.build(df.copy(), df.copy(), os.path.join(output_dir, "informe_verificacion_siniestros"))


✅ Excel generado en: c:\Desarrollo\strix-system\reports\incident\test\integration\output\informe_verificacion_siniestros.xlsx


In [3]:
from reports.incident.adapters.map_adapter import MapAdapter
from core.services.event_service import EventService

event_service = EventService(uow)
incident_service = IncidentService(uow)


#from core.adapters.map_adapter import MapAdapter
from datetime import timedelta

mapas_generados = []


for idx in df.index:
    try:
        id_domain = int(df.loc[idx, "id_domain"])
        fecha = df.loc[idx, "fecha_ocurrencia"]

        start_date = fecha.normalize()
        end_date = start_date + timedelta(days=1) - timedelta(seconds=1)

        print("⏱️", fecha)
        recorrido = event_service.get_events_by_domain_and_date(id_domain, start_date, end_date)
        

        if recorrido is not None and isinstance(recorrido, list) and len(recorrido) > 0:
            recorrido_df = pd.DataFrame(recorrido)
            recorrido_df["domain"] = df.loc[idx, "domain"]  
            mapa = MapAdapter.build_map(idx, df, recorrido_df,output_dir)
            if mapa:
                mapas_generados.append(os.path.join(output_dir, mapa))
        else:
            print(f"⚠️ No hay recorrido para ID {id_domain}")
    except Exception as e:
        print(f"⚠️ Error generando mapa del siniestro ID {idx}: {e}")

print("🗺️ Mapas generados:", mapas_generados)




✅ Conexión a S3 establecida con éxito.
⏱️ 2024-12-28 00:00:00
🚨 ENTER: Se abrió una nueva conexión SQLAlchemy
desde=2024-12-28 00:00:00  , hasta= 2024-12-28 23:59:59
✅ Mapa generado: mapa_siniestro_AG459TC_2024_12_28.html
⏱️ 2024-02-12 00:00:00
🚨 ENTER: Se abrió una nueva conexión SQLAlchemy
desde=2024-02-12 00:00:00  , hasta= 2024-02-12 23:59:59
✅ Mapa generado: mapa_siniestro_AA015FC_2024_02_12.html
🗺️ Mapas generados: ['c:\\Desarrollo\\strix-system\\reports\\incident\\test\\integration\\output\\mapa_siniestro_AG459TC_2024_12_28.html', 'c:\\Desarrollo\\strix-system\\reports\\incident\\test\\integration\\output\\mapa_siniestro_AA015FC_2024_02_12.html']
